It might be useful to run 'pip3 install &lt;library&gt; --upgrade" to make sure you have latest versions of pandas, matplotlib, etc.

In [1]:
import gzip            
import io              
import numpy as np
import pandas as pd    
import pymysql.cursors 
import rdflib
from rdflib import Namespace
import urllib.request  

import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [2]:
# Connect to the database
# I will email you the file sebastia_adsq_params.txt . It's just a four line text file.
# Put that in directory and the following line will read it.
db_params = open("mikepnyu_database_params.txt").read().split()

In [3]:
# Connect to the database

connection = pymysql.connect(host=db_params[0],
                             user=db_params[1],
                             password=db_params[2],
                             db=db_params[3],
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

## Libanius_Letters

In [4]:
sql = "select * from Libanius_Letters"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)

In [5]:
newcol = df['Letter_ID']
df = df.assign(idd = newcol)
df.rename(index=str, columns={"idd": "Letter_Label"}, inplace = True)

In [6]:
pseudo_triples = pd.melt(df,id_vars='Letter_ID')

In [7]:
for t in pseudo_triples.iterrows():
    s = 'ex:%s' % t[1]['Letter_ID']
    p = 'ex:%s' % t[1]['variable']
    if t[1]['variable'] in ['Date_Min','Date_Max','Destination_ID','Recipient_ID','Carrier_ID']:
        print('%s %s %s' % (s,p,t[1]['value']))
    else:
        print('%s %s "%s"' % (s,p,t[1]['value']))

ex:F0040 ex:Date_Min 358
ex:F0075 ex:Date_Min 359
ex:F0083 ex:Date_Min 359
ex:F0095 ex:Date_Min 359
ex:F0099 ex:Date_Min 360
ex:F0108 ex:Date_Min 359
ex:F0119 ex:Date_Min 357
ex:F0142 ex:Date_Min 359
ex:F0154 ex:Date_Min 360
ex:F0156 ex:Date_Min 360
ex:F0158 ex:Date_Min 360
ex:F0159 ex:Date_Min 360
ex:F0166 ex:Date_Min 360
ex:F0175 ex:Date_Min 360
ex:F0215 ex:Date_Min 360
ex:F0220 ex:Date_Min 360
ex:F0242 ex:Date_Min 359
ex:F0251 ex:Date_Min 360
ex:F0252 ex:Date_Min 360
ex:F0253 ex:Date_Min 360
ex:F0265 ex:Date_Min 361
ex:F0293 ex:Date_Min 361
ex:F0298 ex:Date_Min 361
ex:F0308 ex:Date_Min 361
ex:F0315 ex:Date_Min 357
ex:F0332 ex:Date_Min 357
ex:F0336 ex:Date_Min 357
ex:F0339 ex:Date_Min 358
ex:F0348 ex:Date_Min 358
ex:F0354 ex:Date_Min 357
ex:F0359 ex:Date_Min 357
ex:F0361 ex:Date_Min 357
ex:F0362 ex:Date_Min 358
ex:F0366 ex:Date_Min 358
ex:F0386 ex:Date_Min 358
ex:F0399 ex:Date_Min 355
ex:F0438 ex:Date_Min 355
ex:F0458 ex:Date_Min 355
ex:F0459 ex:Date_Min 355
ex:F0482 ex:Date_Min 356


In [8]:
g = rdflib.Graph()
RDF = Namespace("http://example.org/people/")
EXid = Namespace("http://example.org/id/")
EXvocab = Namespace("http://example.org/vocab/")

In [9]:
pseudo_triples = pd.melt(df,id_vars='Letter_ID').sort_values('Letter_ID')

for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['Letter_ID'])
    p = rdflib.URIRef('http://example.org/vocab/%s' % t[1]['variable'])

    if t[1]['variable'] in ['Date_Min','Date_Max']:
        o = rdflib.Literal(int(t[1]['value']))
        g.add((s,p,o))
#    elif t[1]['variable'] in ['pleiades']:
#        o = rdflib.URIRef(t[1]['value'])
#        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [10]:
result = g.query(
        """SELECT * WHERE {
            ?s ?p ?o .
                } LIMIT 10""")
pd.DataFrame(result.bindings)

,o,p,s
0,None,http://example.org/vocab/Destination_ID,http://example.org/id/F0108
1,F1392,http://example.org/vocab/Letter_Label,http://example.org/id/F1392
2,360,http://example.org/vocab/Date_Min,http://example.org/id/F0220
3,363,http://example.org/vocab/Date_Min,http://example.org/id/F1357
4,P0035,http://example.org/vocab/Recipient_ID,http://example.org/id/F0099
5,P0036,http://example.org/vocab/Recipient_ID,http://example.org/id/F1399
6,361,http://example.org/vocab/Date_Max,http://example.org/id/F0251
7,,http://example.org/vocab/Carrier_ID,http://example.org/id/F0799
8,P0028,http://example.org/vocab/Recipient_ID,http://example.org/id/F0339
9,F0656,http://example.org/vocab/Letter_Label,http://example.org/id/F0656


## Libanius_People

In [11]:
sql = "select * from Libanius_People"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)

In [12]:
#df.rename(index=str, columns={"id": "label"}, inplace = True)
newcol = df['Person_ID']
df = df.assign(idd = newcol)
df.rename(index=str, columns={"idd": "Person_Label"}, inplace = True)

In [13]:
pseudo_triples = pd.melt(df,id_vars='Person_ID').sort_values('Person_ID')

In [14]:
pseudo_triples = pd.melt(df,id_vars='Person_ID').sort_values('Person_ID')
pseudo_triples
for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['Person_ID'])
    p = rdflib.URIRef('http://example.org/vocab/%s' % t[1]['variable'])
    
    if t[1]['variable'] in ['']:
        o = rdflib.Literal(float(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [15]:
result = g.query(
        """SELECT * WHERE {
            ?s ?p ?o .
                } LIMIT 10""")
pd.DataFrame(result.bindings)

,o,p,s
0,None,http://example.org/vocab/Destination_ID,http://example.org/id/F0108
1,F1392,http://example.org/vocab/Letter_Label,http://example.org/id/F1392
2,P0013,http://example.org/vocab/Person_Label,http://example.org/id/P0013
3,P0077,http://example.org/vocab/Person_Label,http://example.org/id/P0077
4,Florentius ii,http://example.org/vocab/BLZG_ID,http://example.org/id/P0077
5,363,http://example.org/vocab/Date_Min,http://example.org/id/F1357
6,P0035,http://example.org/vocab/Recipient_ID,http://example.org/id/F0099
7,P0036,http://example.org/vocab/Recipient_ID,http://example.org/id/F1399
8,,http://example.org/vocab/Parent,http://example.org/id/P0017
9,361,http://example.org/vocab/Date_Max,http://example.org/id/F0251


## Libanius_Careers

In [16]:
sql = "select * from Libanius_Careers"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)

In [17]:
#df.rename(index=str, columns={"id": "label"}, inplace = True)
#newcol = df['Career_ID']
#df = df.assign(idd = newcol)
#df.rename(index=str, columns={"idd": "Person_Label"}, inplace = True)

In [18]:
pseudo_triples = pd.melt(df,id_vars='Career_ID').sort_values('Career_ID')
pseudo_triples
for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['Career_ID'])
    p = rdflib.URIRef('http://example.org/vocab/%s' % t[1]['variable'])
    
    if t[1]['variable'] in ['Date_Min','Date_Max']:
        o = rdflib.Literal(int(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [19]:
result = g.query(
        """SELECT * WHERE {
            ?s <http://example.org/vocab/Position> ?o .
                } LIMIT 10""")
pd.DataFrame(result.bindings)

,o,s
0,Comes rei privatae,http://example.org/id/C00051
1,Urban Prefect,http://example.org/id/C00015
2,Governor,http://example.org/id/C00018
3,Praetorian Prefect,http://example.org/id/C00049
4,Assessor,http://example.org/id/C00026
5,Governor,http://example.org/id/C00029
6,Governor,http://example.org/id/C00022
7,Notary,http://example.org/id/C00048
8,Vicar,http://example.org/id/C00003
9,Consul,http://example.org/id/C00039


## Libanius_Mentions

In [20]:
sql = "select * from Libanius_Mentions"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)

In [21]:
#df.rename(index=str, columns={"id": "label"}, inplace = True)
#newcol = df['Career_ID']
#df = df.assign(idd = newcol)
#df.rename(index=str, columns={"idd": "Person_Label"}, inplace = True)

In [22]:
pseudo_triples = pd.melt(df,id_vars='Mention_ID').sort_values('Mention_ID')
pseudo_triples
for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['Mention_ID'])
    p = rdflib.URIRef('http://example.org/vocab/%s' % t[1]['variable'])
    
    if t[1]['variable'] in ['Date_Min','Date_Max']:
        o = rdflib.Literal(int(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [23]:
result = g.query(
        """SELECT * WHERE {
            ?s <http://example.org/vocab/Letter_ID> ?o .
                } LIMIT 10""")
pd.DataFrame(result.bindings)

,o,s
0,F0685,http://example.org/id/M00041
1,F0121,http://example.org/id/M00037
2,F0044,http://example.org/id/M00034
3,F0781,http://example.org/id/M00057
4,F1222,http://example.org/id/M00023
5,F1223,http://example.org/id/M00024
6,F1121,http://example.org/id/M00045
7,F0359,http://example.org/id/M00005
8,F0651,http://example.org/id/M00015
9,F1342,http://example.org/id/M00046


## Libanius_Types

In [24]:
sql = "select * from Libanius_Types"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)

In [25]:
#df.rename(index=str, columns={"id": "label"}, inplace = True)
newcol = df['Type_ID']
df = df.assign(idd = newcol)
df.rename(index=str, columns={"idd": "Type_Label"}, inplace = True)

In [26]:
pseudo_triples = pd.melt(df,id_vars='Type_ID').sort_values('Type_ID')
pseudo_triples
for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['Type_ID'])
    p = rdflib.URIRef('http://example.org/vocab/%s' % t[1]['variable'])
    
    if t[1]['variable'] in ['Date_Min','Date_Max']:
        o = rdflib.Literal(int(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [27]:
result = g.query(
        """SELECT * WHERE {
            ?s <http://example.org/vocab/Type_Label> ?o .
                } LIMIT 10""")
pd.DataFrame(result.bindings)

,o,s
0,T01,http://example.org/id/T01
1,T05,http://example.org/id/T05
2,T04,http://example.org/id/T04
3,T06,http://example.org/id/T06
4,T03,http://example.org/id/T03
5,T02,http://example.org/id/T02


## Libanius_Linking_Types

In [28]:
sql = "select * from Libanius_Linking_Types"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)

In [29]:
#df.rename(index=str, columns={"id": "label"}, inplace = True)
newcol = df['Letter_ID']
df = df.assign(idd = newcol)
df.rename(index=str, columns={"idd": "Letter_Label"}, inplace = True)

In [30]:
pseudo_triples = pd.melt(df,id_vars='Letter_ID').sort_values('Letter_ID')
pseudo_triples
for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['Letter_ID'])
    p = rdflib.URIRef('http://example.org/vocab/%s' % t[1]['variable'])
    
    if t[1]['variable'] in ['Date_Min','Date_Max']:
        o = rdflib.Literal(int(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [31]:
result = g.query(
        """SELECT * WHERE {
            ?s <http://example.org/vocab/Letter_Label> ?o .
                } LIMIT 10""")
pd.DataFrame(result.bindings)

,o,s
0,F1392,http://example.org/id/F1392
1,F0632,http://example.org/id/F0632
2,F0621,http://example.org/id/F0621
3,F1411,http://example.org/id/F1411
4,F0154,http://example.org/id/F0154
5,F1170,http://example.org/id/F1170
6,F0656,http://example.org/id/F0656
7,F0354,http://example.org/id/F0354
8,F0095,http://example.org/id/F0095
9,F0075,http://example.org/id/F0075


## Libanius_Destinations

In [32]:
sql = "select * from Libanius_Destinations"

with connection.cursor() as cursor:

    cursor.execute(sql)
    names = [ x[0] for x in cursor.description]
    result = cursor.fetchall()

df = pd.DataFrame(result, columns = names)

In [33]:
#df.rename(index=str, columns={"id": "label"}, inplace = True)
newcol = df['Destination_ID']
df = df.assign(idd = newcol)
df.rename(index=str, columns={"idd": "Destination_Label"}, inplace = True)

In [34]:
pseudo_triples = pd.melt(df,id_vars='Destination_ID').sort_values('Destination_ID')
pseudo_triples
for t in pseudo_triples.iterrows():
    s = rdflib.URIRef('http://example.org/id/%s' % t[1]['Destination_ID'])
    p = rdflib.URIRef('http://example.org/vocab/%s' % t[1]['variable'])
    
    if t[1]['variable'] in ['Latitude','Longitude']:
        o = rdflib.Literal(float(t[1]['value']))
        g.add((s,p,o))
    elif t[1]['variable'] in ['']:
        o = rdflib.URIRef(t[1]['value'])
        g.add((s,p,o))
    else:
        o = rdflib.Literal(str(t[1]['value']))
        g.add((s,p,o))

In [35]:
result = g.query(
        """SELECT * WHERE {
            ?s <http://example.org/vocab/Latitude> ?o .
                } LIMIT 10""")
pd.DataFrame(result.bindings)

,o,s
0,39.96282667,http://example.org/id/Ancyra
1,39.321242,http://example.org/id/Galatia
2,31.94964296,http://example.org/id/Arabia
3,31.19824567,http://example.org/id/Alexandria
4,31.25,http://example.org/id/Palestina Prima
5,31.928333,http://example.org/id/Palestine
6,29.08574333,http://example.org/id/Egypt
7,41.00746276,http://example.org/id/Constantinople
8,39.25,http://example.org/id/Cappadocia
9,36.914956,http://example.org/id/Tarsus


## Interesting Queries

I have adapted the queries that I previously used in SQL to SPARQL, since those queries already achieved most of what I wanted to do and show with my data. 

In [36]:
#Shows the basic information about the letters, their recipients and their carriers. Links Libanius_Letters with Libanius_People

query1 = g.query(
        """SELECT ?1Letter_Label ?2Recipient_Name ?3Recipient_PLRE ?Date_Min ?Date_Max ?Destination_ID 
        ?4Carrier_Name ?5Carrier_PLRE WHERE {
            ?letter <http://example.org/vocab/Recipient_ID> ?reci ;
                    <http://example.org/vocab/Letter_Label> ?1Letter_Label ;
                    <http://example.org/vocab/Date_Min> ?Date_Min ;
                    <http://example.org/vocab/Date_Max> ?Date_Max ;
                    <http://example.org/vocab/Destination_ID> ?Destination_ID ;
                    <http://example.org/vocab/Carrier_ID> ?cari .
            ?recipient <http://example.org/vocab/Person_Name> ?2Recipient_Name ;
                <http://example.org/vocab/Person_Label> ?reci ;
                <http://example.org/vocab/PLRE_ID> ?3Recipient_PLRE .
            OPTIONAL { ?carrier <http://example.org/vocab/Person_Name> ?4Carrier_Name ;
                <http://example.org/vocab/Person_Label> ?cari ;
                <http://example.org/vocab/PLRE_ID> ?5Carrier_PLRE . }
                } ORDER BY ?2Recipient_Name LIMIT 15 """)

pd.DataFrame(query1.bindings)

,1Letter_Label,2Recipient_Name,3Recipient_PLRE,4Carrier_Name,5Carrier_PLRE,Date_Max,Date_Min,Destination_ID
0,F0752,Acacius,Acacius 8,NaN,NaN,362,362,None
1,F0651,Acacius,Acacius 8,NaN,NaN,361,361,Galatia
2,F0657,Acacius,Acacius 8,NaN,NaN,361,361,None
3,F0732,Acacius,Acacius 8,NaN,NaN,362,362,Galatia
4,F0658,Acacius,Acacius 8,NaN,NaN,361,361,None
5,F1514,Acacius,Acacius 8,NaN,NaN,365,365,Cappadocia
6,F1174,Acacius,Acacius 8,NaN,NaN,364,364,Cappadocia
7,F0735,Acacius,Acacius 7,NaN,NaN,362,362,Cilicia
8,F0622,Acacius,Acacius 8,NaN,NaN,361,361,None
9,F0298,Acacius,Acacius 8,Themistius,Themistius 2,361,361,Galatia


In [37]:
#Improvement over the last query, which adds career information about the recipient. 
#Links Libanius_Letters with Libanius_People and with Libanius Careers

query2 = g.query(
        """SELECT ?1Letter_Label ?2Recipient_Name ?3Recipient_PLRE ?Date_Min ?Date_Max ?Destination_ID 
        ?4Carrier_Name ?5Carrier_PLRE ?position WHERE {
            ?letter <http://example.org/vocab/Recipient_ID> ?reci ;
                    <http://example.org/vocab/Letter_Label> ?1Letter_Label ;
                    <http://example.org/vocab/Date_Min> ?Date_Min ;
                    <http://example.org/vocab/Date_Max> ?Date_Max ;
                    <http://example.org/vocab/Destination_ID> ?Destination_ID ;
                    <http://example.org/vocab/Carrier_ID> ?cari .
            ?recipient <http://example.org/vocab/Person_Name> ?2Recipient_Name ;
                <http://example.org/vocab/Person_Label> ?reci ;
                <http://example.org/vocab/PLRE_ID> ?3Recipient_PLRE .
            OPTIONAL { ?career <http://example.org/vocab/Person_ID> ?reci ;
                               <http://example.org/vocab/Position> ?position ;
                               <http://example.org/vocab/Date_Min> ?career_min ;
                               <http://example.org/vocab/Date_Max> ?career_max . 
                               FILTER ( ?Date_Min >= ?career_min ) .
                               FILTER ( ?Date_Max <= ?career_max ) . }
            OPTIONAL { ?carrier <http://example.org/vocab/Person_Name> ?4Carrier_Name ;
                <http://example.org/vocab/Person_Label> ?cari ;
                <http://example.org/vocab/PLRE_ID> ?5Carrier_PLRE . }
                } ORDER BY ?2Recipient_Name LIMIT 15 """)

pd.DataFrame(query2.bindings)

,1Letter_Label,2Recipient_Name,3Recipient_PLRE,4Carrier_Name,5Carrier_PLRE,Date_Max,Date_Min,Destination_ID,position
0,F0752,Acacius,Acacius 8,NaN,NaN,362,362,None,Governor
1,F0651,Acacius,Acacius 8,NaN,NaN,361,361,Galatia,Governor
2,F0657,Acacius,Acacius 8,NaN,NaN,361,361,None,Governor
3,F0732,Acacius,Acacius 8,NaN,NaN,362,362,Galatia,Governor
4,F0658,Acacius,Acacius 8,NaN,NaN,361,361,None,Governor
5,F1514,Acacius,Acacius 8,NaN,NaN,365,365,Cappadocia,Comes Domorum
6,F1174,Acacius,Acacius 8,NaN,NaN,364,364,Cappadocia,Comes Domorum
7,F0735,Acacius,Acacius 7,NaN,NaN,362,362,Cilicia,NaN
8,F0622,Acacius,Acacius 8,NaN,NaN,361,361,None,Governor
9,F0298,Acacius,Acacius 8,Themistius,Themistius 2,361,361,Galatia,Governor


In [38]:
#This query shows all the letters in which a certain individual has been mentioned or that he received.
#Links Libanius_Letters with Libanius_People, Libanius Careers and Libanius_Mentions -> many-to-many relationship

query3 = g.query(
        """SELECT ?1Person_Name ?2Person_PLRE ?3Letter_Label ?4Date_Min ?5Date_Max 
        (CONCAT(?position, " of ", ?location) AS ?6Position)
        WHERE {
            ?mention <http://example.org/vocab/Person_ID> ?per ;
                    <http://example.org/vocab/Letter_ID> ?3Letter_Label .
            ?letter <http://example.org/vocab/Letter_Label> ?3Letter_Label ;
                    <http://example.org/vocab/Date_Min> ?4Date_Min ;
                    <http://example.org/vocab/Date_Max> ?5Date_Max .
            ?person <http://example.org/vocab/Person_Label> ?per ;
                    <http://example.org/vocab/Person_Name> ?1Person_Name ;
                    <http://example.org/vocab/PLRE_ID> ?2Person_PLRE .
                    FILTER ( ?2Person_PLRE = "Acacius 8" )
            OPTIONAL { ?career <http://example.org/vocab/Person_ID> ?per ;
                               <http://example.org/vocab/Position> ?position ;
                               <http://example.org/vocab/Location> ?location ;
                               <http://example.org/vocab/Date_Min> ?career_min ;
                               <http://example.org/vocab/Date_Max> ?career_max . 
                               FILTER ( ?4Date_Min >= ?career_min ) .
                               FILTER ( ?5Date_Max <= ?career_max ) . }
                } ORDER BY ?4Date_Min""")

pd.DataFrame(query3.bindings)

,1Person_Name,2Person_PLRE,3Letter_Label,4Date_Min,5Date_Max,6Position
0,Acacius,Acacius 8,F0621,361,361,Governor of Galatia
1,Acacius,Acacius 8,F0658,361,361,Governor of Galatia
2,Acacius,Acacius 8,F0298,361,361,Governor of Galatia
3,Acacius,Acacius 8,F0617,361,361,Governor of Galatia
4,Acacius,Acacius 8,F0622,361,361,Governor of Galatia
5,Acacius,Acacius 8,F0651,361,361,Governor of Galatia
6,Acacius,Acacius 8,F0308,361,361,Governor of Galatia
7,Acacius,Acacius 8,F0657,361,361,Governor of Galatia
8,Acacius,Acacius 8,F0752,362,362,Governor of Galatia
9,Acacius,Acacius 8,F0732,362,362,Governor of Galatia


In [39]:
#Shows the letter types
#Links Libanius_Letters with Libanius_Types through Libanius_Linking_Types -> many-to-many relationship

query4 = g.query(
        """SELECT ?1Letter_Label ?2Type_Name ?3Type_Description WHERE {
            ?letter <http://example.org/vocab/Recipient_ID> ?reci ;
                    <http://example.org/vocab/Letter_Label> ?1Letter_Label ;
                    <http://example.org/vocab/Type_ID> ?type .
            ?typology <http://example.org/vocab/Type_Label> ?type ;
                      <http://example.org/vocab/Type_Name> ?2Type_Name ;
                      <http://example.org/vocab/Description> ?3Type_Description .
                } ORDER BY ?1Letter_Label LIMIT 15 """)

pd.DataFrame(query4.bindings)

,1Letter_Label,2Type_Name,3Type_Description
0,F0040,Recommendation,A letter that recommends an individual for a p...
1,F0075,Recommendation,A letter that recommends an individual for a p...
2,F0083,Favour,A letter whose purpose is to obtain a favour f...
3,F0095,Introduction/Reference,A letter that introduces an individual to anot...
4,F0099,Recommendation,A letter that recommends an individual for a p...
5,F0108,Introduction/Reference,A letter that introduces an individual to anot...
6,F0119,Personal,A letter whose purpose is to maintain his rela...
7,F0142,Personal,A letter whose purpose is to maintain his rela...
8,F0154,Favour,A letter whose purpose is to obtain a favour f...
9,F0154,Recommendation,A letter that recommends an individual for a p...


In [40]:
#Shows all the letters of recommendation

query5 = g.query(
        """SELECT ?Letter_Label ?Type_Name WHERE {
            ?letter <http://example.org/vocab/Recipient_ID> ?reci ;
                    <http://example.org/vocab/Letter_Label> ?Letter_Label ;
                    <http://example.org/vocab/Type_ID> ?type .
            ?typology <http://example.org/vocab/Type_Label> ?type ;
                      <http://example.org/vocab/Type_Name> ?Type_Name .
                      FILTER (?Type_Name = "Recommendation")
                } ORDER BY ?Letter_Label LIMIT 15 """)

pd.DataFrame(query5.bindings)

,Letter_Label,Type_Name
0,F0040,Recommendation
1,F0075,Recommendation
2,F0099,Recommendation
3,F0154,Recommendation
4,F0252,Recommendation
5,F0339,Recommendation
6,F0348,Recommendation
7,F0482,Recommendation
8,F0560,Recommendation
9,F0563,Recommendation
